<a href="https://colab.research.google.com/github/mwmachado/AzureFunctions/blob/main/TioPat_facil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
# libraries
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [74]:
# Fetch lottery data
jogos = requests.get('https://loteriascaixa-api.herokuapp.com/api/lotofacil')
jogos = pd.DataFrame(jogos.json()).set_index('concurso')[["dezenas", "acumulou"]][::-1]

In [75]:
# Expand 'dezenas' and get 'acumulou' as numpy array
dezenas = np.array(jogos.dezenas.to_list()).astype(int)
acumulou = np.array(jogos.acumulou).reshape(-1,1)

# Binarize 'dezenas'
mlb = MultiLabelBinarizer()
dezenas = mlb.fit_transform(dezenas)

# Create dataframe
df = np.hstack((dezenas, acumulou))

# Split train and test data
X_train = df[:-2]
y_train = df[1:-1, :-1]
x_test = df[-2]
y_test = df[-1, :-1]

In [76]:
# Reshape input for LSTM [samples, time steps, features]
X = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Create the LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(100, input_shape=(X.shape[1], X.shape[2]))))
#model.add(Dropout(0.2))
#model.add(Dense(30, activation='sigmoid'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

# Compile the model with a different optimizer and loss function
model.compile(loss='binary_crossentropy', optimizer='adam')

# Train the model
model.fit(X, y_train, epochs=100, batch_size=60)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.6813
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.6727
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.6720
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.6713
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.6710
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6711
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6706
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6701
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6697
Epoch 10/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6699
Epoch 11/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6692
Epoch 12/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6693
Epoch 13/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6689
Epoch 14/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6689
Epoch 15/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.6692
Epoch 16/100
5

In [77]:
# Prediction
prediction = model.predict(x_test.reshape((1, 1, -1)))

# Post-process the prediction
winner_game = np.array(mlb.inverse_transform(y_test.reshape(1, -1)))
prob = pd.Series(prediction.flatten(), index=range(prediction.shape[0],prediction.shape[1]+1)).sort_values(ascending=False).reset_index()
prob.index = prob.index + 1
prob.columns = ['dezenas', 'probability']

# Get top 10 predictions
top = prob.head(int(winner_game.shape[1] * 5/3))
top

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step


,dezenas,probability
1,3,0.800725
2,17,0.799673
3,11,0.795278
4,2,0.765569
5,24,0.732538
6,9,0.731866
7,22,0.727502
8,18,0.714481
9,1,0.681195
10,7,0.656478


In [78]:
top.dezenas.to_list()

[3,
 17,
 11,
 2,
 24,
 9,
 22,
 18,
 1,
 7,
 13,
 6,
 19,
 23,
 14,
 25,
 15,
 20,
 5,
 12,
 10,
 16,
 4,
 8,
 21]

In [79]:
# Generate combinations based on predicted probabilities
combinations = [
  np.random.choice(prob['dezenas'], size=winner_game.shape[1], replace=False, p=prob['probability'].to_numpy()/prob['probability'].to_numpy().sum())
  for _ in range(10)
] + [
  np.random.choice(top['dezenas'], size=winner_game.shape[1], replace=False, p=top['probability'].to_numpy()/top['probability'].to_numpy().sum())
  for _ in range(10)
]
combinations.append(prob[:winner_game.shape[1]].dezenas.to_numpy())
combinations = np.array(combinations)

In [80]:
# Calculate win probabilities
win_prob = prob[prob['dezenas'].isin(winner_game.flatten())]
win_prob

,dezenas,probability
6,9,0.731866
7,22,0.727502
8,18,0.714481
9,1,0.681195
11,13,0.626735
12,6,0.601957
13,19,0.600965
15,14,0.566009
16,25,0.557718
17,15,0.539782


In [81]:
# Evaluate combinations
results = np.isin(combinations, winner_game).sum(axis=1)
results

array([ 8,  8,  9,  8, 11,  6,  7,  9,  9, 10, 11, 10, 11,  6,  9,  8,  9,
        6,  9,  8,  8])

In [82]:
win_prob.index

Index([6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 24], dtype='int64')

In [83]:
# Define custom evaluation metric
metric = lambda x: np.where(x <= winner_game.shape[1], 0, x).mean()
metric(win_prob.index)

array([ 0,  0,  0,  0,  0,  0,  0,  0, 16, 17, 18, 20, 21, 22, 24])

In [84]:
# Get the best result
results.max()

11

In [85]:
mlb.inverse_transform(y_test.reshape(1, -1))

[(1, 6, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 22, 25)]